# Supervised Learning for Dog Adoption Analysis

**Student Assignment: Machine Learning Analysis**

## Problem Description

This project aims to create a supervised learning solution that demonstrates and describes the EDA procedure and analysis (model building and training) for Random Forest for dog adoption. This assignment uses the dog adoption dataset from Kaggle: https://www.kaggle.com/datasets/thedevastator/adoptable-dogs-in-the-us?resource=download and takes the first 10,000 rows. 

**Objective**: Build a Random Forest classifier to analyze dog adoption patterns, perform comprehensive EDA, and demonstrate supervised learning techniques for understanding adoption likelihood based on dog characteristics for a dataset of "Adoptable dogs" that I have created.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

In [ ]:
# Load the dataset
df = pd.read_csv('/dataset/allDogDescriptions_first10k.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic dataset information
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData types:")
df.dtypes

In [ ]:
# Missing values analysis
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing_data, 'Percentage': missing_percent})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print("Missing Values:")
print(missing_df)

In [ ]:
# Analyze key features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age distribution
df['age'].value_counts().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Age Distribution')
axes[0,0].tick_params(axis='x', rotation=45)

# Size distribution
df['size'].value_counts().plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Size Distribution')
axes[0,1].tick_params(axis='x', rotation=45)

# Sex distribution
df['sex'].value_counts().plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Sex Distribution')
axes[1,0].tick_params(axis='x', rotation=45)

# Top breeds
df['breed_primary'].value_counts().head(10).plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Top 10 Primary Breeds')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Create processed dataset
df_processed = df.copy()

# Create age groups
def categorize_age_simple(age):
    if age in ['Baby', 'Young', 'Adult']:
        return 'Young'
    else:
        return 'Old'

df_processed['age_simple'] = df_processed['age'].apply(categorize_age_simple)
df_processed['target'] = df_processed['sex'] + '_' + df_processed['age_simple']

# Select features
features = ['size', 'breed_primary', 'color_primary', 'fixed', 'house_trained', 'shots_current']
available_features = [f for f in features if f in df_processed.columns]

X = df_processed[available_features].copy()
y = df_processed['target']

print(f"Target distribution: {y.value_counts()}")

In [ ]:
# Handle missing values and encoding
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].fillna('Unknown')
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    else:
        X[col] = X[col].astype(int)

print(f"Before outlier removal - Target distribution:")
print(y.value_counts())

# Remove classes with only 1 member (outliers) to avoid stratification errors
class_counts = y.value_counts()
classes_to_remove = class_counts[class_counts == 1].index
if len(classes_to_remove) > 0:
    print(f"\nRemoving classes with only 1 member (outliers): {list(classes_to_remove)}")
    mask = ~y.isin(classes_to_remove)
    X = X[mask]
    y = y[mask]
    print(f"Removed {len(classes_to_remove)} outlier classes")
else:
    print("\nNo single-member classes found to remove")

print(f"\nAfter outlier removal - Target distribution:")
print(y.value_counts())
print(f"\nFinal feature matrix shape: {X.shape}")
X.head()

## 4. Random Forest Model Training

In [ ]:
# Debug: Check for single-member classes before splitting
print("Checking target distribution before train_test_split:")
class_counts = y.value_counts()
print(class_counts)
single_member_classes = class_counts[class_counts == 1]
if len(single_member_classes) > 0:
    print(f"\nFound single-member classes: {list(single_member_classes.index)}")
    print("Removing these classes to fix stratification error...")
    mask = ~y.isin(single_member_classes.index)
    X = X[mask]
    y = y[mask]
    print(f"\nAfter removal - Final target distribution:")
    print(y.value_counts())
    print(f"Final dataset shape: {X.shape}")
else:
    print("No single-member classes found.")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

print(feature_importance)

## 5. Breed Prediction Analysis

In [ ]:
# Breed prediction setup
top_breeds = df_processed['breed_primary'].value_counts().head(10).index
breed_df = df_processed[df_processed['breed_primary'].isin(top_breeds)].copy()

breed_features = ['age', 'sex', 'size', 'color_primary']
available_breed_features = [f for f in breed_features if f in breed_df.columns]

X_breed = breed_df[available_breed_features].copy()
y_breed = breed_df['breed_primary']

# Encode features
for col in X_breed.columns:
    if X_breed[col].dtype == 'object':
        X_breed[col] = X_breed[col].fillna('Unknown')
        le = LabelEncoder()
        X_breed[col] = le.fit_transform(X_breed[col].astype(str))

print(f"Breed prediction dataset size: {len(breed_df)}")
print(f"Top breeds: {list(top_breeds)}")

In [ ]:
# Train breed model
X_breed_train, X_breed_test, y_breed_train, y_breed_test = train_test_split(
    X_breed, y_breed, test_size=0.2, random_state=42, stratify=y_breed
)

rf_breed = RandomForestClassifier(n_estimators=100, random_state=42)
rf_breed.fit(X_breed_train, y_breed_train)

y_breed_pred = rf_breed.predict(X_breed_test)
breed_accuracy = accuracy_score(y_breed_test, y_breed_pred)

print(f"Breed Prediction Accuracy: {breed_accuracy:.4f}")

## 6. Size Classification Analysis

In [ ]:
# Size classification setup
size_df = df_processed.dropna(subset=['size']).copy()
size_features = ['age', 'sex', 'breed_primary', 'color_primary']
available_size_features = [f for f in size_features if f in size_df.columns]

X_size = size_df[available_size_features].copy()
y_size = size_df['size']

# Encode features
for col in X_size.columns:
    if X_size[col].dtype == 'object':
        X_size[col] = X_size[col].fillna('Unknown')
        le = LabelEncoder()
        X_size[col] = le.fit_transform(X_size[col].astype(str))

print(f"Size distribution: {y_size.value_counts()}")

In [ ]:
# Train size model
X_size_train, X_size_test, y_size_train, y_size_test = train_test_split(
    X_size, y_size, test_size=0.2, random_state=42, stratify=y_size
)

rf_size = RandomForestClassifier(n_estimators=100, random_state=42)
rf_size.fit(X_size_train, y_size_train)

y_size_pred = rf_size.predict(X_size_test)
size_accuracy = accuracy_score(y_size_test, y_size_pred)

print(f"Size Classification Accuracy: {size_accuracy:.4f}")
print(f"\nClassification Report:")
print(classification_report(y_size_test, y_size_pred))

## 7. Top 3 Sex-Age Combinations

In [ ]:
# Analyze sex-age combinations
target_counts = df_processed['target'].value_counts()
target_props = df_processed['target'].value_counts(normalize=True)

print("=== TOP 3 SEX-AGE COMBINATIONS ===")
for i, (combo, count) in enumerate(target_counts.head(3).items()):
    prop = target_props[combo]
    print(f"{i+1}. {combo.replace('_', ' ')}: {count} dogs ({prop*100:.1f}%)")

# Visualize
plt.figure(figsize=(10, 6))
target_counts.plot(kind='bar')
plt.title('Sex-Age Combinations Distribution')
plt.xlabel('Combination')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Model Performance Summary

In [ ]:
# Summary of all models
print("=== MODEL PERFORMANCE SUMMARY ===")
print(f"Sex-Age Classification Accuracy: {accuracy:.4f}")
print(f"Breed Prediction Accuracy: {breed_accuracy:.4f}")
print(f"Size Classification Accuracy: {size_accuracy:.4f}")

# Create summary visualization
models = ['Sex-Age', 'Breed', 'Size']
accuracies = [accuracy, breed_accuracy, size_accuracy]

plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=accuracies)
plt.title('Random Forest Model Performance Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')
plt.tight_layout()
plt.show()

## Conclusions

### EDA Findings
- I used EDA to perform an analysis on the source dataset. I analyzed for dog charecteristics and distributions, identified key patterns in age, sex, size and breed and performed handling for missing values and schema mismatches.

### Random Forest Applications
I used random forest to analyse the dataset. This allowed me to demonstrate three Random Forest applications:

1. **Sex-Age Classification**: Demographic pattern analysis of age /sex
2. **Breed Prediction**: Identification and verification of dog breeds
3. **Size Classification**: Which would allow for better planning of where to send dogs (shelters that can take small dogs vs large dogs, how to set up new shelter areas etc.)

### Key Insights
- Random Forest proved effective across multiple classification tasks
- Feature importance analysis revealed key predictive characteristics

## Next Step: Can I use this to classify the adoptability of new dogs?

## 9. Adoptability Prediction Test on Unadopted Dogs

In [ ]:
# I created a random selection of 10 dogs here based on charecteristics taken from the source dataset but setting dogs to status = not_adopted.
test_df = pd.read_csv('test_unadopted_dogs.csv')
print("Test Dataset - Unadopted Dogs:")
print(test_df)
print(f"\nTest dataset shape: {test_df.shape}")

In [ ]:
# Prepare test data for prediction using same preprocessing
test_df['age_simple'] = test_df['age'].apply(categorize_age_simple)
test_df['target'] = test_df['sex'] + '_' + test_df['age_simple']

# Prepare features matching training data
X_test_new = test_df[available_features].copy()

# Encode features using same approach as training
for col in X_test_new.columns:
    if X_test_new[col].dtype == 'object':
        X_test_new[col] = X_test_new[col].fillna('Unknown')
        le = LabelEncoder()
        # Combine training and test values to handle unseen categories
        combined_values = list(X[col].unique()) + list(X_test_new[col].unique())
        le.fit(combined_values)
        X_test_new[col] = le.transform(X_test_new[col].astype(str))
    else:
        X_test_new[col] = X_test_new[col].astype(int)

print("Processed test features:")
print(X_test_new.head())

In [ ]:
# Make predictions using the trained Random Forest model
predictions = rf_model.predict(X_test_new)
prediction_probs = rf_model.predict_proba(X_test_new)

# Create results dataframe
results_df = test_df[['age', 'sex', 'size', 'breed_primary', 'special_needs']].copy()
results_df['Predicted_Category'] = predictions
results_df['Adoptability_Score'] = prediction_probs.max(axis=1)

print("Adoptability Predictions for Unadopted Dogs:")
print(results_df)

In [ ]:
# Create Viz
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Predicted categories distribution
category_counts = pd.Series(predictions).value_counts()
category_counts.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Predicted Sex-Age Categories for Unadopted Dogs')
ax1.set_xlabel('Predicted Category')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=45)

# Plot 2: Individual adoptability confidence scores
dog_names = [f'Dog {i+1}' for i in range(len(results_df))]
colors = ['red' if score < 0.5 else 'orange' if score < 0.7 else 'green' 
          for score in results_df['Adoptability_Score']]

bars = ax2.bar(dog_names, results_df['Adoptability_Score'], color=colors)
ax2.set_title('Adoptability Confidence Scores by Dog')
ax2.set_xlabel('Individual Dogs')
ax2.set_ylabel('Model Confidence Score')
ax2.set_ylim(0, 1)
ax2.tick_params(axis='x', rotation=45)

# Add score labels on bars
for bar, score in zip(bars, results_df['Adoptability_Score']):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nConfidence Score Legend:")
print("🔴 Red: Low confidence (< 0.5) - May need special attention")
print("🟠 Orange: Medium confidence (0.5-0.7) - Moderate adoption potential")
print("🟢 Green: High confidence (> 0.7) - High adoption potential")

In [ ]:
# Detailed analysis and recommendations
print("=== DETAILED ADOPTABILITY ANALYSIS ===")
for i, row in results_df.iterrows():
    print(f"\nDog {i+1}: {row['age']} {row['sex']} {row['breed_primary']}")
    print(f"  Size: {row['size']}, Special Needs: {row['special_needs']}")
    print(f"  Predicted Category: {row['Predicted_Category']}")
    print(f"  Confidence Score: {row['Adoptability_Score']:.3f}")
    
    if row['Adoptability_Score'] > 0.7:
        recommendation = "High adoption potential - prioritize for adoption events"
    elif row['Adoptability_Score'] > 0.5:
        recommendation = "Moderate potential - consider targeted marketing"
    else:
        recommendation = "Needs special attention - may require behavioral/medical support"
    
    print(f"  Recommendation: {recommendation}")

# Summary statistics and actionable insights
high_potential = (results_df['Adoptability_Score'] > 0.7).sum()
medium_potential = ((results_df['Adoptability_Score'] > 0.5) & 
                   (results_df['Adoptability_Score'] <= 0.7)).sum()
low_potential = (results_df['Adoptability_Score'] <= 0.5).sum()

print(f"\n=== SHELTER MANAGEMENT SUMMARY ===")
print(f"High adoption potential: {high_potential} dogs ({high_potential/len(results_df)*100:.1f}%)")
print(f"Medium adoption potential: {medium_potential} dogs ({medium_potential/len(results_df)*100:.1f}%)")
print(f"Need special attention: {low_potential} dogs ({low_potential/len(results_df)*100:.1f}%)")

In [ ]:
# Create summary visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart of adoptability distribution
labels = ['High Potential', 'Medium Potential', 'Need Attention']
sizes = [high_potential, medium_potential, low_potential]
colors = ['green', 'orange', 'red']
ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Adoptability Distribution for Unadopted Dogs')

# Bar chart comparing all model accuracies
models = ['Sex-Age\nClassification', 'Breed\nPrediction', 'Size\nClassification']
accuracies = [accuracy, breed_accuracy, size_accuracy]
bars = ax2.bar(models, accuracies, color=['lightblue', 'lightgreen', 'lightcoral'])
ax2.set_title('Random Forest Model Performance Summary')
ax2.set_ylabel('Accuracy')
ax2.set_ylim(0, 1)

# Add accuracy labels
for bar, acc in zip(bars, accuracies):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

##Conclusions
- I demonstrated the use of a trained random forest model to categorize uncategorized dogs for adoption based on key features.
- 